In [23]:
# Walk through the data folder, read the comments and write it to text_data.txt file
import os
from os import walk

current_dir = os.getcwd() # First, get the path of the working directory
path1 =current_dir+'\\data\\negative'
path2 =current_dir+'\\data\\positive'

_, _, list1 = next(walk(path1))
_, _, list2 = next(walk(path2))

filename_with_path=current_dir+'\\data\\'+'Text_Data.txt'
f1= open(filename_with_path,"w+")
f1.close()

for i in range(len(list1)):
 f1 = open(filename_with_path,"a+")

 data=""
    
 f = open(path1+'\\'+list1[i])
 data = f.read()
    
 f.close()

 f = open(path2+'\\'+list2[i])
    
 data=data+f.read()
 f.close()
 f1.write(data)
 f1.close()
### This code creates the dataset "Text_Data.txt" under ./data
path_to_file = current_dir+'\\data\\'+'Text_Data.txt'

In [8]:
# Get the file. If the code above executed, next time you just need to start here since tetx_data.txt is created
import os
from os import walk

current_dir = os.getcwd()
path_to_file = current_dir+'\\data\\'+'Text_Data.txt'

In [9]:
# read the file
text_data_file = open(path_to_file,"r+")
text_data = text_data_file.read()

In [10]:
len(text_data)

7786004

In [11]:
text_data[0:100]

'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \n'

In [12]:
# Number of different characters
"".join(sorted(set(text_data.lower()))) 

'\x05\n\x12\x13\x14\x16 !"#$%&\'()*+,-./0123456789:;=>?@[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~'

In [13]:
# tokenize the text
from tensorflow import keras
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text_data)

In [14]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['w o r n t']

In [15]:
# number of distinct characters => max_id = 39 in this case.
max_id = len(tokenizer.word_index)

# total number of characters
dataset_size = tokenizer.document_count 
dataset_size

7786004

In [18]:
import numpy as np
np.random.seed(42)

# encode the full text so each character is represented by its ID
[encoded] = np.array(tokenizer.texts_to_sequences([text_data])) - 1
[encoded]

[array([18, 10,  5, ..., 23,  0, 24])]

In [19]:
len(encoded)

7786004

In [21]:
import tensorflow as tf
tf.random.set_seed(42)

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [22]:
n_steps = 100

# target = input shifted 1 character ahead
window_length = n_steps + 1 

In [23]:
# Prepare the dataset for stateless RNN
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

dataset = dataset.map((lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)))
dataset = dataset.prefetch(1)

In [24]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 74) (32, 100)


Stateless RNN:

In [ ]:
# Stateless model
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     # no dropout in stateful RNN (https://github.com/ageron/handson-ml2/issues/32)
                     # dropout=0.2, recurrent_dropout=0.2,
                     ),
    keras.layers.GRU(128, return_sequences=True,
                     # dropout=0.2, recurrent_dropout=0.2
                    ),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

# Compile and fit for 10 epochs.
# About training these models, I couldn't train them on jupyter notebook because I didn't figure out how to
# enable GPU on Jupyter Notebook. Without GPU it takes too long to train.
# I copied the code and trained these models on Google Colab and they worked well
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=50)

In [26]:
# Function to preprocess the text before making the prediction on it
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

# Function to predict the next character using temperature t
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

# Function to predict the next n (100 in default) characters using temperature t
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

Stateless RNN predictions:

In [ ]:
# Make predictions of next 100 characters on first character a
print(complete_text("a", temperature=0.2))

In [ ]:
# Make predictions of next 100 characters on first character s
print(complete_text("s", temperature=0.2))

In [ ]:
# Make predictions of next 100 characters on first character t
print(complete_text("t", temperature=0.2))

Statefull RNN:

In [ ]:
# Prepare the dataset for the statefull RNN
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
# Statefull model
stateFullModel = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
# Compile and fit
stateFullModel.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
stateFullModel.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                   callbacks=[ResetStatesCallback()])

In [ ]:
# Create a statless copy so the model can predict on batches with different size
stateless_copy = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

stateless_copy.build(tf.TensorShape([None, None, max_id]))
stateless_copy.set_weights(stateFullModel.get_weights())

model = stateless_copy

Statefull RNN predictions:

In [ ]:
# Make predictions of next 100 characters on first character a
print(complete_text("a", temperature=0.2))

In [ ]:
# Make predictions of next 100 characters on first character s
print(complete_text("s", temperature=0.2))

In [ ]:
# Make predictions of next 100 characters on first character t
print(complete_text("t", temperature=0.2))